In [10]:
import csv
import pandas as pd
import re

In [11]:
sw = pd.read_csv('isw.csv')
sw

,col0,col1,col2,col3,col4
0,24-02-2022,Russia-Ukraine Warning Update: Initial Russian...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...","\r\nMason Clark, George Barros, and Kateryna S..."
1,25-02-2022,Russia-Ukraine Warning Update: Russian Offensi...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Key Takeaways\r\nRussian forces carried out ad...
2,26-02-2022,Russia-Ukraine Warning Update: Russian Offensi...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Ukrainian resistance remains remarkably effect...
3,27-02-2022,Russia-Ukraine Warning Update: Russian Offensi...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",\r\nThe Russian military has likely recognized...
4,28-02-2022,"Russian Offensive Campaign Assessment, Februar...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",The Russian military is reorganizing its milit...
...,...,...,...,...,...
331,21-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",The Ukrainian defense of Bakhmut is likely a s...
332,22-01-2023,"Russian Offensive Campaign Assessment, Januar...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...","January 22, 8:30 pm ET\r\nISW is publishing an..."
333,23-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...","January 23, 9:45pm ET \r\nUkrainian intelligen..."
334,24-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",A coalition of NATO member states reportedly w...


In [12]:
sw.rename(columns={'col0': 'date', 'col1': 'title', 'col2': 'url', 'col3': 'html', 'col4': 'main_text'}, inplace=True)

In [13]:
sw.tail(3)

,date,title,url,html,main_text
333,23-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...","January 23, 9:45pm ET \r\nUkrainian intelligen..."
334,24-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",A coalition of NATO member states reportedly w...
335,25-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Russian forces may be engaging in limited spo...


In [14]:
sw[sw["main_text"].isna()]

,date,title,url,html,main_text
273,24-11-2022,NaN,NaN,NaN,NaN
304,25-12-2022,NaN,NaN,NaN,NaN
311,01-01-2023,NaN,NaN,NaN,NaN


In [15]:
#convert all the data in the column "main_text" to rows
sw['main_text'] = sw['main_text'].astype(str)

In [16]:
sw.dtypes

date         object
title        object
url          object
html         object
main_text    object
dtype: object

In [17]:
#sw[sw["date"].str.endswith("08-2022")]

In [18]:
#remove authors
texts_auth = [
    "\r\nMason Clark, George Barros, and Kateryna Stepanenko\r\nFebruary 24, 3:00 pm EST\r\n",
    "Karolina Hird, Kateryna Stepanenko, and Mason Clark\r\nJune 9, 6:45 pm ET\r\n",
    "Frederick W. Kagan\r\nSeptember 25, 6 pm ET\r\n ",
    "Kateryna Stepanenko and Frederick W. Kagan\r\nOctober 9, 9:35 pm ET\r\n",
    "\r\n\r\nKarolina Hird, Grace Mappes, George Barros, Katherine Lawlor, and Mason Clark \r\nJanuary 10, 8:00 pm ET\r\n "
]

for text in texts_auth:
    sw['main_text'] = sw['main_text'].str.replace(text, '', regex=False)

In [19]:
#remove the dates
date_pattern = r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+(?:\d{1,2}:\d{2}\s*(?:am|pm)?\s*ET?|\d+(?:\s*pm|am)?\s*ET?)\b'
sw['main_text'] = sw['main_text'].apply(lambda x: re.sub(date_pattern, '', x).lstrip())

specific_pattern = r'\bDecember 8, 6:40 PM ET\b'
sw['main_text'] = sw['main_text'].apply(lambda x: re.sub(specific_pattern, '', x).lstrip())

In [20]:
#remove symbols [1], [2], ... [n]
unnecessary_numeration = r'\[\d{1,2}\]'
sw['main_text'] = sw['main_text'].apply(lambda x: re.sub(unnecessary_numeration, '', x))

In [21]:
#remove symbols "\r\n" and "\xa0"
sw['main_text'] = sw['main_text'].replace({'\r\n': ' ', '\xa0': ' '}, regex=True)

In [22]:
#unnecessary text in rows
un_pattern = r'We do not report in detail on Russian war crimes because those activities are well-covered in Western media and do not directly affect the military operations we are assessing and forecasting. We will continue to evaluate and report on the effects of these criminal activities on the Ukrainian military and population and specifically on combat in Ukrainian urban areas. We utterly condemn these Russian violations of the laws of armed conflict, Geneva Conventions, and humanity even though we do not describe them in these reports.'
un_cases = sw[sw['main_text'].str.contains(un_pattern, regex=True)]
un_cases

,date,title,url,html,main_text
41,06-04-2022,"Russian Offensive Campaign Assessment, April 6...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Russian forces continued to redeploy forces to...
42,07-04-2022,"Russian Offensive Campaign Assessment, April 7...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Russian proxy forces claimed to have captured ...
43,08-04-2022,"Russian Offensive Campaign Assessment, April 8...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Ukrainian forces retain control of defensive p...
44,09-04-2022,"Russian Offensive Campaign Assessment, April 9...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Special Edition: Russian Military Capabilities...
45,10-04-2022,"Russian Offensive Campaign Assessment, April 1...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Russian forces made territorial gains in Mariu...
...,...,...,...,...,...
330,20-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Head of the Chechen Republic Ramzan Kadyrov co...
331,21-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",The Ukrainian defense of Bakhmut is likely a s...
333,23-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Ukrainian intelligence assessed that Russian f...
334,24-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",A coalition of NATO member states reportedly w...


In [23]:
#remove unnecessary text in rows
unwanted_text = "We do not report in detail on Russian war crimes because those activities are well-covered in Western media and do not directly affect the military operations we are assessing and forecasting. We will continue to evaluate and report on the effects of these criminal activities on the Ukrainian military and population and specifically on combat in Ukrainian urban areas. We utterly condemn these Russian violations of the laws of armed conflict, Geneva Conventions, and humanity even though we do not describe them in these reports."

sw['main_text'] = sw['main_text'].replace(unwanted_text, '', regex=True)

In [24]:
#checkup
un_pattern = r'We do not report in detail on Russian war crimes because those activities are well-covered in Western media and do not directly affect the military operations we are assessing and forecasting. We will continue to evaluate and report on the effects of these criminal activities on the Ukrainian military and population and specifically on combat in Ukrainian urban areas. We utterly condemn these Russian violations of the laws of armed conflict, Geneva Conventions, and humanity even though we do not describe them in these reports.'
un_cases = sw[sw['main_text'].str.contains(un_pattern, regex=True)]
un_cases

,date,title,url,html,main_text


In [25]:
#remove "Key Takeaways" text
key_text = "Key Takeaways"
sw['main_text'] = sw['main_text'].replace(key_text, '', regex=True)

In [26]:
#remove extra spaces and rpresent each row by a "\n"
sw['main_text'] = sw['main_text'].apply(lambda x: "\n" + re.sub(r'\s+', ' ', x.strip()))

In [27]:
sw.tail(5)

,date,title,url,html,main_text
331,21-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",\nThe Ukrainian defense of Bakhmut is likely a...
332,22-01-2023,"Russian Offensive Campaign Assessment, Januar...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",\nISW is publishing an abbreviated campaign up...
333,23-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",\nUkrainian intelligence assessed that Russian...
334,24-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",\nA coalition of NATO member states reportedly...
335,25-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",\nRussian forces may be engaging in limited sp...


In [30]:
sw.loc[0, 'main_text']

'\nRussian President Vladimir Putin began a large-scale invasion of Ukraine on February 24 likely aimed at full regime change and the occupation of Ukraine. His claimed objective to “demilitarize” and “de-nazify” Ukraine is a transparent cover for an unprovoked war of aggression to occupy a neighboring state. Putin and Kremlin media continue to deny that the Russian invasion is a war, instead describing it as a special military operation. Putin’s messaging is likely aimed at a domestic Russian audience, which the Kremlin has not fully prepared for the costs of a war against Ukraine. Russian officials and state media have been denying and mocking Western warnings of the impending Russian invasion for months and as recently as February 23. Russian forces remain much larger and more capable than Ukraine’s conventional military. Russia will likely defeat Ukrainian regular military forces and secure their territorial objectives at some point in the coming days or weeks if Putin is determine

In [29]:
sw.to_csv('updated_isw.csv', index=False)